In [1]:
import ndjson
import os
from multiprocessing import Pool, cpu_count
import textract
from utils.csv import remove_stopwords
from functools import partial

In [2]:
data_path = 'data/threads'
text_path = 'data/threads_text'

In [3]:
def pdf_extract_text(file,root, out_dir):
    name,_ = os.path.splitext(file)
    text = textract.process(os.path.join(root,file))
    text = remove_stopwords(text, remove_numbers=True)
    text = text.replace("'","").replace('"','').lower()
    with open(os.path.join(out_dir,name+'.txt'),'w') as f:
        f.write(text)

In [4]:
def walk_pdf_extract_text(in_dir, out_dir):
    if not os.path.exists(out_dir):
        os.mkdir(out_dir) 
    root,_,files = next(os.walk(in_dir))
    with Pool(cpu_count()) as p:
        f = partial(pdf_extract_text,root=root, out_dir=out_dir)
        p.map(f,files)

In [5]:
walk_pdf_extract_text(data_path,text_path)

In [6]:
def make_huge_json(in_dir):
    dicts = []
    root,_,files = next(os.walk(in_dir))
    for file in files:
        thread_id,_ = os.path.splitext(file)
        with open(os.path.join(in_dir,file)) as f:
            text = f.read()
        dicts.append({'thread_id':thread_id, 'text':text})
    return dicts

In [7]:
d = make_huge_json('data/threads_text')

In [8]:
with open('data/threads.json','w') as f:
    ndjson.dump(d,f)